In [2]:
from sklearn.metrics import make_scorer

In [21]:
def rf_to_lr(temp):
    temp['log_flow'] = np.log(temp['flow'].astype(float))
    temp['log_Conduct'] = np.log(temp['Conduct'].astype(float))
    temp = temp.drop(["flow", "Conduct"],axis =1)
    return temp
        

In [22]:
def lr_to_rf(temp):
    temp['flow'] = np.exp(temp['log_flow'].astype(float))
    temp['Conduct'] = np.exp(temp['log_Conduct'].astype(float))
    temp = temp.drop(["log_flow", "log_Conduct"],axis =1)
    return temp

In [23]:
def algo_runner(ss, name_regressor):
    list_feat = ss.columns.drop(predict)
    features = ss[list_feat]
    label = ss[[predict]]

    X = np.array(features)
    y = np.array(label)

    arr_r2 = []
    arr_RMSE = []

    if name_regressor == "rf":
        model = RandomForestRegressor(n_jobs = -1)
    elif name_regressor == "lr":
        model = linear_model.LinearRegression(fit_intercept=False, normalize='deprecated', copy_X=True,
                                              n_jobs=None, positive=True)

    kf = KFold(n_splits=5)
    for train_index, test_index in kf.split(X):
        X_train, X_test, y_train, y_test = X[train_index], X[test_index], y[train_index], y[test_index]
        model.fit(X_train,  y_train.ravel())
        predictions = model.predict(X_test)

        temp = r2_score(y_test, predictions)
        arr_r2.append(temp)

        temp2 = mean_squared_error(y_test, predictions, squared=False)
        arr_RMSE.append(temp2)

        r2 = np.average(arr_r2)
        rmse = np.average(arr_RMSE)
            
    print("Rsquared", predict, "\n", r2)
    print('RMSE ', predict, '\n', rmse)

In [24]:
def blank_dataframe(df_feat, predict):
    boss_list = [x for x in df_feat.columns] 
    boss_list.append((predict))
    boss_list.append(("pd_" + predict))

    trial2 = pd.DataFrame(columns= boss_list)
    return trial2

In [25]:
def comp_cleaned_df(df_feat, cleaned_df):
    sexxxy = df_feat.reset_index()[["Datum"]]
    cleaned_df = cleaned_df.reset_index().drop("index",axis = 1)
    trial3 = pd.concat([sexxxy, cleaned_df], axis = 1)
    trial3 = trial3.set_index("Datum")
    return trial3

In [26]:
def count(df, column_string):
    count_array = []
    column_name = []
    i = 0
    while i < len(df.columns):
        count_array.append(df.describe().iloc[0,i])
        column_name.append(df.describe().columns[i])
        i = i+1
        
    xx = pd.DataFrame({'Parameter':column_name, column_string : count_array})
    xx= xx.set_index("Parameter")
    return xx

In [3]:
def rmse_manual(y, y_pred):
    return mean_squared_error(y, y_pred, squared=False)

scorer_rmse = make_scorer(rmse_manual, greater_is_better=True)

In [27]:
def plotvariale(df, variable):
    plt.figure(figsize=(14,8))
    plt.subplot(1,2,1) ## means 1 row , 2 columns and 1st plot
    df[variable].hist(bins=30)
    
    ## QQ plot
    plt.subplot(1,2,2)
    stats.probplot(df[variable], dist="norm", plot=pylab)
    plt.show()

In [28]:
def Data_count(pre, post):
    Data_count = pd.concat([pre, post], axis=1)
    Data_count["Loss"] = Data_count["Pre Count"] - Data_count["Post Count"]
    Data_count["Percentage Loss"] = Data_count["Loss"]*100/Data_count["Pre Count"] 
    Data_count.dropna()
    return Data_count

In [29]:
def plot_kahl_in_two(df, perimeter, title):
    temp1 = df.loc[:"2019-08-31"]
    temp2 = df.loc["2019-09-01":]
    fig = px.scatter(temp1.reset_index(), x ="Datum", y= perimeter , title =title , width=1000, height=400)
    fig2 = px.scatter(temp2.reset_index(), x ="Datum", y= perimeter , title =title , width=1000, height=400)
    fig.update_xaxes(rangeslider_visible = True)
    fig.show()
    fig2.update_xaxes(rangeslider_visible = True)
    fig2.show()

In [30]:
def plot_kahl(df, parameter, title):
    fig = px.scatter(df.reset_index(), x ="Datum", y= parameter , title = title, width=1000, height=400)
    fig.update_xaxes(rangeslider_visible = True)
    fig.show()

In [33]:
def order_of_features(df, diction):
    df = df[["feature_names"]]
    rows = len(df.index)
    array = []
    i =1
    while i <= rows:
        test = df.iloc[(i-1),0]
        array.append(test)
        i = i+1

    distinct = []
    for i in array:
        for j in i:
            distinct.append(j)

    final_distinct = list(dict.fromkeys(distinct))
    
    final_array = []
    for i in final_distinct:
        final_array.append(diction.get(i))
        
    return final_array

In [32]:
def column_dict(df):
    ss = df.columns
    diction = {}
    i = 0
    while i < len(ss):
        diction[str(i)] = ss[i]
        i = i+1
    return diction